A prototype for technical question/answer using OpenAI Anthropic models, and Ollama. 

In [ ]:
import os
import gradio as gr 
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display
from scraper import fetch_website_contents

In [4]:
MODEL_GPT = 'gpt-4.1-mini'
MODEL_LLAMA = 'llama3:latest'
MODEL_CLAUDE = 'claude-sonnet-4-5-20250929'

In [5]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)


In [ ]:
openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
ollama = OpenAI(api_key="ollama", base_url=OLLAMA_BASE_URL)

openai = OpenAI()


In [ ]:
def stream_gpt(company_name, context, history, message):
    relevant_system_message = f"""
    You are a helpful assistant for {company_name}. Answer technical questions based on the context provided from their documentation.
    Context from {company_name} documentation:
    {context}
    """
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_claude(company_name, context, history, message):
    relevant_system_message = f"""
    You are a helpful assistant for {company_name}. Answer technical questions based on the context provided from their documentation.
    Context from {company_name} documentation:
    {context}
    """
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = anthropic.chat.completions.create(
        model=MODEL_CLAUDE,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [52]:
def stream_ollama(company_name, context, history, message):
    relevant_system_message = f"""
    You are a helpful assistant for {company_name}. Answer technical questions based on the context provided from their documentation.
    Context from {company_name} documentation:
    {context}
    """
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages)

    yield response.choices[0].message.content


In [53]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [54]:
def stream_model(model, company_name, context, history, message):
    if model=="GPT":
        result = stream_gpt(company_name, context, history, message)
    elif model=="Claude":
        result = stream_claude(company_name, context, history, message)
    elif model=="Ollama":
        result = stream_ollama(company_name, context, history, message)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
def chat(message, history, company_name, doc_url, model):
    if not company_name or not doc_url:
        yield "Please provide both the company name and link to documentation above before asking questions."
        return
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    context = fetch_website_contents(doc_url)
    result = stream_model(model, company_name, context, history, message)
    yield from result

In [ ]:
company_name_input = gr.Textbox(label="Company name:", placeholder="e.g. Hugging Face")
doc_url_input = gr.Textbox(
    label="Link to technical documentation",
    placeholder="https://docs.example.com or https://example.com/docs",
    lines=1
)
model_selector = gr.Dropdown(["GPT", "Claude", "Ollama"], label="Select model", value="GPT")

gr.ChatInterface(
    fn=chat,
    type="messages",
    additional_inputs=[company_name_input, doc_url_input, model_selector],
    title="Documentation Q&A"
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


Received message: Explain web component
Received history: []
Received company_name: Salesforce
Received doc_url: https://developer.salesforce.com/docs/platform/lightning-component-reference/guide
Received model: Ollama
